In [ ]:
#| default_exp inspecttools

# inspecttools
> Solveit tools for inspecting the symbol table and importing modules

This module provides **LLM tools** for Solveit to dynamically inspect source code, types, and module capabilities. Functions take *string* arguments (dotted symbol paths) rather than Python objects because LLM tool interfaces can only pass serializable values—not live Python references.

In [ ]:
#| export
import inspect, re, sys, ast, builtins
from importlib import import_module
from dialoghelper import add_msg
from tracefunc import tracefunc

In [ ]:
#| export
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = inspect.currentframe().f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
#| export
def doimport(mod: str):
    "Import a module into the caller's global namespace"
    g = _find_frame_dict('__msg_id')
    import_module(mod)
    g[mod.split('.')[0]] = import_module(mod.split('.')[0])

`doimport` lets the LLM dynamically import modules by name. Here we import `fastcore.utils` and verify it's available.

In [ ]:
doimport('fastcore.utils')
fastcore.__version__

'1.11.3'

In [ ]:
#| export
_last = None

def resolve(
    sym: str  # Dotted symbol path, with optional [n] indexing, e.g. "module.attr.subattr[1]" or "_last" for previous result
):
    """Resolve a dotted symbol string to its Python object, with optional [n] indexing. Sets global `_last` to the resolved object for chaining.
    Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist"""
    global _last
    if (sym := sym.strip()) == '_last': return _last
    g = _find_frame_dict('__msg_id')
    if match := re.match(r'^(\w+)\[(\d+)\]$', sym):
        attr, idx = match.groups()
        parts, _last = ['_last'], _last[int(idx)] if attr == '_last' else g[attr][int(idx)]
    else: parts = re.split(r'\.(?![^\[]*\])', sym)
    obj = _last if parts[0] == '_last' else g[parts[0]]
    for part in parts[1:]:
        match = re.match(r'(\w+)\[(\d+)\]$', part)
        if match:
            attr, idx = match.groups()
            obj = getattr(obj, attr)[int(idx)]
        else: obj = getattr(obj, part)
    _last = obj
    return obj

`resolve` navigates dotted paths like `"a.argfirst"` to reach the actual Python object.

In [ ]:
a = fastcore.utils.L(1)
resolve('a.argfirst')

<bound method L.argfirst of [1]>

It also sets `_last` for chaining. Since it's used internally by all `get*` tools in this module, the tools all set `_last` too.

In [ ]:
_last

<bound method L.argfirst of [1]>

It works on both objects and classes:

In [ ]:
resolve('fastcore.utils.L.argfirst')

<function fastcore.foundation.L.argfirst(self: fastcore.foundation.L, f, negate=False)>

In [ ]:
#| export
def symsrc(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get the source code for a symbol.

    Examples:

    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `getnth("module.dispatcher.funcs", n) then symsrc("_last")`"""
    obj = resolve(sym)
    source = inspect.getsource(obj)
    filename = inspect.getfile(obj)
    return f"File: {filename}\n\n{source}"

`symsrc` retrieves the source code of any symbol by path—essential for letting the LLM understand how functions work.

In [ ]:
print(symsrc('a.argfirst'))

File: /Users/jhoward/aai-ws/fastcore/fastcore/foundation.py

@patch
@curryable
def argfirst(self:L, f, negate=False):
    "Return index of first matching item"
    if negate: f = not_(f)
    return first(i for i,o in self.enumerate() if f(o))



In [ ]:
#| export
def showsrc(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    "Create a note to show the user (and following LLM prompts) the source of `sym`, following `symsrc` rules"
    res = symsrc(sym)
    add_msg(f'```python\n{res}\n```')
    return {'success': 'Message has been added to dialog successfully'}

`showsrc` is like `symsrc` but adds the information as a note message.

In [ ]:
#| export
def gettype(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get the type of a symbol and set `_last`.

    Examples:

    - `gettype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `gettype("_last")` -> type of previous result"""
    return type(resolve(sym))

`gettype` returns the type of a symbol—useful for the LLM to understand what kind of object it's dealing with.

In [ ]:
gettype('a.argfirst')

method

In [ ]:
#| export
def getdir(
    sym: str,  # Dotted symbol path or "_last" for previous result
    exclude_private: bool=False # Filter out attrs starting with "_"
):
    """Get dir() listing of a symbol's attributes and set `_last`. E.g: `getdir("sympy.Interval")` -> `['__add__', '__and__', ...]`"""
    res = dir(resolve(sym))
    if not exclude_private: return res
    return [o for o in res if o[0]!='_']

`getdir` lists all attributes of an object (i.e it calls `dir()`. Here we filter out private names to see the public API of an `L` list.

In [ ]:
' '.join(getdir('a', exclude_private=True))

'accumulate append argfirst argwhere attrgot batched clear combinations compress concat copy copy count cycle dropwhile enumerate extend filter flatmap flatten groupby index insert itemgot items map map_dict map_first map_zip map_zipwith pairwise partition permutations pop product range reduce remove renumerate reverse rstarargfirst rstarargwhere rstardropwhile rstarfilter rstarmap rstarpartition rstarreduce rstarsorted rstartakewhile setattrs shuffle sort sorted split splitlines starargfirst starargwhere stardropwhile starfilter starmap starpartition starreduce starsorted startakewhile sum takewhile unique val2idx zip zipwith'

In [ ]:
#| export
def getval(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get repr of a symbol's value and set `_last`.

    Examples:
    
    - `getval("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `getval("some_dict.keys")` -> `dict_keys([...])`"""
    return repr(resolve(sym))

`getval` returns the `repr()` of a symbol's value—handy for inspecting data without needing to execute arbitrary code.

In [ ]:
getval('a')

'[1]'

In [ ]:
#| export
def getnth(
    sym: str,  # Dotted symbol path to a dict or object with .values()
    n: int     # Index into the values (0-based)
):
    """Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `getnth("dispatcher.funcs", 12)` -> 13th registered function
    - `getnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler"""
    global _last
    _last = list(resolve(sym).values())[n]
    return _last

`getnth` extracts the nth value from a dict (or anything with `.values()`).

In [ ]:
handlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))
getnth('handlers', 0)

<function __main__.<lambda>(x)>

Combined with `_last`, this lets the LLM drill into registries of handlers/dispatchers and then inspect their source.

In [ ]:
symsrc('_last')

'File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_22794/2745627804.py\n\nhandlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))\n'

In [ ]:
#| export
def symlen(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    "Returns the length of the given symbol"
    return len(resolve(sym))

In [ ]:
#| export
def symslice(
    sym: str,   # Dotted symbol path or "_last" for previous result
    start: int, # Starting index for slice
    end: int    # Ending index for slice
):
    "Returns the contents of the symbol from the given start to the end."
    try: return resolve(sym)[start:end]
    except Exception as e: return f'Error: {e}'

In [ ]:
a = ['a', 'b', 'c', 'd']
symslice('a', 1, 3)

['b', 'c']

On failure we get a str error:

In [ ]:
symslice('resolve', 0, 1)

"Error: 'function' object is not subscriptable"

In [ ]:
#| export
def symsearch(
    sym:str,      # Dotted symbol path or "_last" for previous result
    term:str,     # Search term (exact string or regex pattern)
    regex:bool=True,  # If True, regex search; if False, exact match
    flags:int=0   # Regex flags (e.g., re.IGNORECASE)
):
    """Search contents of symbol, which is assumed to be str for regex, or iterable for non-regex.
    Regex mode returns (match, start, end) tuples; otherwise returns (item, index) tuples"""
    if regex: return str([(m.group(), m.start(), m.end()) for m in re.finditer(term, resolve(sym), flags)])
    else: return str([(x, i) for i, x in enumerate(resolve(sym)) if x == term])

In [ ]:
symsearch('a', 'c', regex=False), symsearch('a', 'z', regex=False)

("[('c', 2)]", '[]')

In [ ]:
text = "The quick brown fox jumps over 3 lazy dogs and 12 cats"

In [ ]:
symsearch('text', r'\d+', regex=True)

"[('3', 31, 32), ('12', 47, 49)]"

In [ ]:
symsearch('text', r'\b[aeiou]\w*', regex=True, flags=re.IGNORECASE)

"[('over', 26, 30), ('and', 43, 46)]"

In [ ]:
#| export
def symset(
    val: str  # Value to assign to _ai_sym
):
    "Set _ai_sym to the given value"
    _find_frame_dict('__msg_id')['_ai_sym'] = val

In [ ]:
symset('Otters are awesome!'); _ai_sym

'Otters are awesome!'

In [ ]:
#| export
def run_code_interactive(
    code:str # Code to have user run
):
    """Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one."""
    add_msg('# Please run this:\n'+code, msg_type='code')
    return {'success': "CRITICAL: Message added to user dialog. STOP IMMEDIATELY. Do NOT call any more tools. Wait for user to run code and respond."}

In [ ]:
#| export
_builtins = set(dir(builtins))

def _collapse(v): return v[0] if len(v) > 0 and all(x == v[0] for x in v) else v
def _process(vars): return {k: _collapse(v) for k,v in vars.items() if k not in _builtins}

def tracetool(
    sym: str,  # Dotted symbol path to callable or "_last" for previous result
    args: list=None,  # Positional args (JSON values passed directly)
    kwargs: dict=None  # Keyword args (JSON values passed directly)
) -> dict:  # Dict, in source code order, mapping source snippets to (hit_count, variables); unchanged vars collapsed to single tuple
    "Trace execution of callable at `sym` with given args/kwargs. Variables are captured AFTER each line executes."
    d = tracefunc(resolve(sym), *(args or []), **(kwargs or {}))
    return {src: (hits, _process(vars)) for src, (hits, vars) in d.items()}

In [ ]:
def demo(n,m='x'):
    total = 0
    for i in range(n): total += i
    return m*total

Try using &`tracetool` to trace `demo(5, m='y')`. Based on the results, tell me how `demo` works.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>
<summary>tracetool(sym=demo, args=[5], kwargs={'m': 'y'})</summary>

```json
{
  "id": "toolu_019cPTthikVG24XJ7u4YjJEd",
  "call": {
    "function": "tracetool",
    "arguments": {
      "sym": "demo",
      "args": "[5]",
      "kwargs": "{'m': 'y'}"
    }
  },
  "result": "{'total = 0': (1, {'total': ('int', '0')}),\n 'for i in range(n):': (6,\n  {'i': [('int', '0'),\n    ('int', '1'),\n    ('int', '2'),\n    ('int', '3'),\n    ('int', '4'),\n    ('int', '4')],\n   'n': ('int', '5'),\n   'range': ('type', \"<class 'range'>\")}),\n 'total += i': (5,\n  {'i': [('int', '0'), ('int', '1'), ('int', '2'), ('int', '3'), ('int', '4')],\n   'total': [('int', '0'),\n    ('int', '1'),\n    ('int', '3'),\n    ('int', '6'),\n    ('int', '10')]}),\n 'return m*total': (1, {'m': ('str', \"'y'\"), 'total': ('int', '10')})}"
}
```

</details>

Here's how `demo` works:

1. **Initializes** `total = 0`
2. **Loops** `for i in range(n)` — with `n=5`, it iterates `i` through 0, 1, 2, 3, 4
3. **Accumulates** `total += i` each iteration, so total becomes 0 → 0 → 1 → 3 → 6 → 10 (the sum 0+1+2+3+4)
4. **Returns** `m * total` — multiplies the string `'y'` by 10, giving `'yyyyyyyyyy'`

So `demo(n, m)` computes the sum of integers from 0 to n-1, then repeats the string `m` that many times. It's essentially `m * sum(range(n))`.

In [ ]:
#| export
def inspect_tool_info():
    add_msg('Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,symlen,symslice,symsearch,symset,run_code_interactive,tracetool]`')

In [ ]:
# inspect_tool_info()

Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,symlen,symslice,symsearch,symset,run_code_interactive]`